# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import seaborn as sns

# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:
cities = pd.read_csv ('..\output_data\cities.csv')
print (cities)

          City      Lat       Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0        Bluff -46.6000  168.3333     61.81        75          93        8.66   
1       Castro -24.7911  -50.0119     62.24        89          64        3.38   
2     Tasiilaq  65.6145  -37.6368     21.36        92          92        4.72   
3       Tutóia  -2.7619  -42.2744     80.78        81          23       12.10   
4       Atuona  -9.8000 -139.0333     79.00        76           3       22.08   
..         ...      ...       ...       ...       ...         ...         ...   
540    Vammala  61.3333   22.9000     13.41        86         100        7.49   
541   Totskoye  52.5248   52.7623     28.60        79         100       16.60   
542    Peniche  39.3558   -9.3811     56.62        78          83        0.89   
543  Mackenzie  55.2999 -123.1698     28.62       100          90        4.61   
544    Sørland  67.6670   12.6934     31.60        83         100        8.55   

    Country        Date  
0

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# NaN values
cities = cities.dropna()
# Convert Poverty Rate to float and store
humidity = cities["Humidity"].astype(float)

In [5]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities[["Lat", "Lng"]].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure(center=(20,0),zoom_level=1.7)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating=False, max_intensity=250,
                                 point_radius=4)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
perfect = cities[(cities["Max Temp"] < 65) & (cities["Humidity"] < 50) & (cities["Wind Speed"] > 6) ]
perfect.dropna()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Atar,20.5169,-13.0499,62.38,30,79,9.66,MR,1638240375
10,Mataura,-46.1927,168.8643,61.48,45,93,6.98,NZ,1638240376
91,Adrar,20.5022,-10.0711,59.90,31,85,11.21,MR,1638240403
110,Khani,41.9563,42.9566,54.97,49,100,11.79,GE,1638240409
276,Sanbu,33.4594,119.1453,49.10,43,5,18.97,CN,1638240463
372,Zhangjiakou,40.8100,114.8794,22.93,46,5,17.74,CN,1638240494
398,Safford,32.8340,-109.7076,57.06,32,1,6.91,US,1638240502
424,Río Gallegos,-51.6226,-69.2181,53.65,40,0,16.11,AR,1638240510
482,Akdağmadeni,39.6603,35.8836,56.89,41,100,19.80,TR,1638240528


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
hotel_df = perfect[["City", "Lat", "Lng", "Country"]].reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Country,Hotel Name
0,Atar,20.5169,-13.0499,MR,
1,Mataura,-46.1927,168.8643,NZ,
2,Adrar,20.5022,-10.0711,MR,
3,Khani,41.9563,42.9566,GE,
4,Sanbu,33.4594,119.1453,CN,
5,Zhangjiakou,40.8100,114.8794,CN,
6,Safford,32.8340,-109.7076,US,
7,Río Gallegos,-51.6226,-69.2181,AR,
8,Akdağmadeni,39.6603,35.8836,TR,


In [31]:
# find the closest hotels to coordinates

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{row.Lat}, {row.Lng}", 
        "radius": "5000",
        "type": "lodging",
        "key": g_key,
    }
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
#    hotelname = response['name']
#     hotel_df["Hotel Name"] = hotel_df.append(hotelname)
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Atar.


NameError: name 'hotel_name' is not defined

In [33]:
response

{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Assign the hotel layer to a variable
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info)

# Add both layers
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
hotel_layer = gmaps.symbol_layer(
    locations,
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

fig